<a href="https://colab.research.google.com/github/ringerH/Hackvita-2.0/blob/main/Object_tracking(SIFT%2BBFMatcher).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import cv2

print("Loading input image...")
input_image = cv2.imread('test2.jpg', cv2.IMREAD_GRAYSCALE)

print("Loading video...")
cap = cv2.VideoCapture('test_video.mp4')

sift = cv2.SIFT_create()

keypoints_input, descriptors_input = sift.detectAndCompute(input_image, None)

bf = cv2.BFMatcher()

occurrences = 0
occurrence_start = 0
occurrence_duration = 0
prev_matches = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video reached.")
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    keypoints_frame, descriptors_frame = sift.detectAndCompute(frame_gray, None)

    matches = bf.knnMatch(descriptors_input, descriptors_frame, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    if len(good_matches) >= 6:
        if not prev_matches:
            occurrence_start = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
            occurrences += 1

        prev_matches = good_matches
        occurrence_duration = (cap.get(cv2.CAP_PROP_POS_MSEC) / 1000) - occurrence_start
    else:
        if prev_matches:
            print(f"Occurrence {occurrences}: Start time: {occurrence_start:.2f}s, Duration: {occurrence_duration:.2f}s")
            prev_matches = []

cap.release()
cv2.destroyAllWindows()


Loading input image...
Loading video...
Occurrence 1: Start time: 0.00s, Duration: 3.64s
Occurrence 2: Start time: 29.76s, Duration: 5.08s
End of video reached.
